In [1]:

import numpy as np
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D,Conv2D,MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator    
from keras.applications import MobileNetV2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score

2025-10-08 04:30:34.807654: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759897835.077782      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759897835.147871      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train='/kaggle/input/fer2013/train'
test='/kaggle/input/fer2013/test'

In [3]:
emotions=['angry', 'happy', 'sad', 'surprise', 'neutral']

In [4]:
import tensorflow as tf


size=(128,128)


train_gen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


train_data = train_gen.flow_from_directory(
    train,
    target_size=size,
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    classes=emotions
)

# Validation data (20%)
val_data = train_gen.flow_from_directory(
    train,
    target_size=size,
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    classes=emotions
)

# Test data (separate unseen dataset)
test_gen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

test_data = test_gen.flow_from_directory(
    test,
    target_size=size,
    batch_size=32,
    class_mode='categorical',
    shuffle=False,
    classes=emotions
)


Found 19341 images belonging to 5 classes.
Found 4835 images belonging to 5 classes.
Found 6043 images belonging to 5 classes.


In [5]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
inputs = Input(shape=(128, 128, 3))
base = MobileNetV2(weights='imagenet', include_top=False, input_tensor=inputs)

x = base.output
x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(5, activation='softmax')(x)

model = Model(inputs, outputs)

model.summary()

/tmp/ipykernel_19/3791187237.py:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base = MobileNetV2(weights='imagenet', include_top=False, input_tensor=inputs)
I0000 00:00:1759897861.607880      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 64, 64,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 64, 64,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 64, 64,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 64, 64,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 64, 64,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 64, 64,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 65, 65,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 32, 32,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 32, 32,    │      2,304 │ block_1_depthwis

 Total params: 3,087,109 (11.78 MB)

 Trainable params: 3,052,613 (11.64 MB)

 Non-trainable params: 34,496 (134.75 KB)

In [6]:
from keras.callbacks import ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

In [7]:

model.fit(
    train_data,
    epochs=40,
    validation_data=val_data,
    callbacks=[early_stopping, reduce_lr]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/40


I0000 00:00:1759897904.144738      80 service.cc:148] XLA service 0x7f37e8002f70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1759897904.145974      80 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1759897907.644144      80 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1759897913.339700      80 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1759897913.536700      80 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


  1/605 ━━━━━━━━━━━━━━━━━━━━ 10:02:10 60s/step - accuracy: 0.1875 - loss: 1.8727

I0000 00:00:1759897925.532069      80 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


443/605 ━━━━━━━━━━━━━━━━━━━━ 36s 223ms/step - accuracy: 0.3857 - loss: 1.5040

E0000 00:00:1759898030.933648      80 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1759898031.129776      80 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


605/605 ━━━━━━━━━━━━━━━━━━━━ 251s 317ms/step - accuracy: 0.4142 - loss: 1.4574 - val_accuracy: 0.4354 - val_loss: 1.5087 - learning_rate: 0.0010
Epoch 2/40
605/605 ━━━━━━━━━━━━━━━━━━━━ 110s 182ms/step - accuracy: 0.6027 - loss: 1.1470 - val_accuracy: 0.5473 - val_loss: 1.3853 - learning_rate: 0.0010
Epoch 3/40
605/605 ━━━━━━━━━━━━━━━━━━━━ 108s 179ms/step - accuracy: 0.6391 - loss: 1.0846 - val_accuracy: 0.5415 - val_loss: 1.4519 - learning_rate: 0.0010
Epoch 4/40
605/605 ━━━━━━━━━━━━━━━━━━━━ 107s 177ms/step - accuracy: 0.6551 - loss: 1.0531 - val_accuracy: 0.5355 - val_loss: 1.4467 - learning_rate: 0.0010
Epoch 5/40
605/605 ━━━━━━━━━━━━━━━━━━━━ 108s 179ms/step - accuracy: 0.6880 - loss: 0.9868 - val_accuracy: 0.6848 - val_loss: 1.0118 - learning_rate: 2.0000e-04
Epoch 6/40
605/605 ━━━━━━━━━━━━━━━━━━━━ 107s 177ms/step - accuracy: 0.7164 - loss: 0.9381 - val_accuracy: 0.6976 - val_loss: 0.9711 - learning_rate: 2.0000e-04
Epoch 7/40
605/605 ━━━━━━━━━━━━━━━━━━━━ 106s 176ms/step - accuracy:

In [8]:
pred2=model.predict(test_data)

189/189 ━━━━━━━━━━━━━━━━━━━━ 35s 168ms/step


In [9]:

print(accuracy_score(test_data.labels, np.argmax(pred2, axis=1)))

0.7565778586794638


In [10]:
from sklearn.metrics import classification_report 
print(classification_report(test_data.classes,np.argmax(pred2, axis=1)))

              precision    recall  f1-score   support

           0       0.69      0.67      0.68       958
           1       0.91      0.87      0.89      1774
           2       0.65      0.67      0.66      1247
           3       0.85      0.84      0.85       831
           4       0.65      0.70      0.67      1233

    accuracy                           0.76      6043
   macro avg       0.75      0.75      0.75      6043
weighted avg       0.76      0.76      0.76      6043



In [11]:
model.save('mobilenetv2.keras')